# syn data

In [609]:
from ctgan import CTGAN
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sdmetrics.reports.single_table import QualityReport


In [610]:
df = pd.read_csv('Data for python.csv', sep=';', header=1, index_col=0, decimal=',')
df = df.fillna(0)
df = df.drop(columns=['Id'])
df = df.drop(columns=['ScandcleftID'])
df = df.drop(columns=['Født'])
df = df.drop(columns=['Materialedato A'])
df = df.drop(columns=['Materialedato B'])
df = df.drop(columns=['Materialedato C'])
df = df.drop(columns=['Total Row Score A'])
df = df.drop(columns=['Total Row Score B'])
df = df.drop(columns=['Total Row Score C'])
df = df.drop(columns=['Goslon Score A'])
df = df.drop(columns=['Goslon Score B'])
df = df.drop(columns=['Goslon Score C'])

In [611]:
A = df[["Anteroposterior 1",
          'Anteroposterior 2',
          'Vertical 1',
          'Vertical 2',
          'Transverse',
          "Crowding ",
          'Spacing',
          "Tooth shape/size\r\n",
          "Pan 2\r\n",
          ]]
B = df[['Anteroposterior 1.1',
          'Anteroposterior',
          'Vertical 1.1',
          'Vertical 2.1',
          'Transverse.1',
          'Crowding .1',
          'Spacing.1',
          'Tooth shape/size\r\n.1',
          'Pan 2\r\n.1',]]


In [612]:
ctgan_A = CTGAN(epochs=5)
ctgan_A.fit(A, A.columns)

ctgan_B = CTGAN(epochs=75)
ctgan_B.fit(B, B.columns)

syn_A= ctgan_A.sample(1500)
syn_B= ctgan_B.sample(20)

In [613]:
ctgan = CTGAN(epochs=128,verbose=True,log_frequency=False)
ctgan.fit(df, df.columns)

syn = ctgan.sample(25000)


Gen. (0.96) | Discrim. (0.13): 100%|██████████| 128/128 [00:10<00:00, 12.16it/s] 


In [614]:
from sdv.metrics.tabular import LogisticDetection, SVCDetection


LogisticDetection.compute(df, syn)

0.4119991177634428

In [615]:
def tis(row):
    x = syn["Anteroposterior 1"]+syn["Anteroposterior 2"]+syn["Vertical 1"]+syn["Vertical 2"]+syn["Transverse"]+syn["Crowding "]+syn["Spacing"]+syn["Tooth shape/size\r\n"]+syn["Pan 2\r\n"]
    y = syn["Anteroposterior 1.1"]+syn["Anteroposterior"]+syn["Vertical 1.1"]+syn["Vertical 2.1"]+syn["Transverse.1"]+syn["Crowding .1"]+syn["Spacing.1"]+syn["Tooth shape/size\r\n.1"]+syn["Pan 2\r\n.1"]
    z = syn["Anteroposterior 1.2"]+syn["Anteroposterior 2.1"]+syn["Vertical 1.2"]+syn["Vertical 2.2"]+syn["Transverse.2"]+syn["Crowding .2"]+syn["Spacing.2"]+syn["Tooth shape/size\r\n.2"]+syn["Pan 2\r\n.2"]
    return x,y, z

In [616]:
def total_row_score_A(row):
    x =  syn_A["Anteroposterior 1"]+syn_A["Anteroposterior 2"]+syn_A["Vertical 1"]+syn_A["Vertical 2"]+syn_A["Transverse"]+syn_A["Crowding "]+syn_A["Spacing"]+syn_A["Tooth shape/size\r\n"]+syn_A["Pan 2\r\n"]
    return x

In [617]:
def Total_row_score_B(row):
    y = syn_B["Anteroposterior 1.1"]+syn_B["Anteroposterior 2.1"]+syn_B["Vertical 1.1"]+syn_B["Vertical 2.1"]+syn_B["Transverse.1"]+syn_B["Crowding .1"]+syn_B["Spacing.1"]+syn_B["Tooth shape/size\r\n.1"]+syn_B["Pan 2\r\n.1"]
    return y

In [618]:
tA ,tb ,tC= tis(syn)
syn["Total Row Score A"] = tA
syn["Total Row Score B"] = tb
syn["Total Row Score C"] = tC

In [619]:
syn = syn.drop(syn[(syn['Total Row Score C'] > syn['Total Row Score B'] ) | (syn['Total Row Score B'] > syn["Total Row Score A"])].index)
syn

,Kassenr,Anteroposterior 1,Anteroposterior 2,Vertical 1,Vertical 2,Transverse,Crowding,Spacing,Tooth shape/size\r\n,Pan 1,...,Vertical 2.2,Transverse.2,Crowding .2,Spacing.2,Tooth shape/size\r\n.2,Pan 1.2,Pan 2\r\n.2,Total Row Score A,Total Row Score B,Total Row Score C
0,8,12.0,0.0,3.0,0.0,6.0,4.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,29.0,17.0,8.0
1,18,12.0,0.0,2.0,1.0,6.0,4.0,0.0,1.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,30.0,14.0,2.0
3,9,9.0,0.0,0.0,0.0,6.0,4.0,2.0,1.0,0.0,...,4.0,8.0,0.0,0.0,0.0,0.0,0.0,26.0,16.0,12.0
4,16,0.0,1.0,0.0,0.0,8.0,4.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,15.0,0.0
5,6,12.0,0.0,3.0,0.0,6.0,4.0,2.0,1.0,0.0,...,4.0,2.0,1.0,0.0,0.0,0.0,0.0,32.0,28.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24991,10,9.0,1.0,2.0,0.0,6.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,16.0,9.0
24992,43,12.0,0.0,3.0,0.0,8.0,3.0,2.0,0.0,0.0,...,0.0,2.0,3.0,0.0,0.0,0.0,0.0,28.0,28.0,5.0
24993,35,12.0,1.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,22.0,17.0,5.0
24996,18,12.0,1.0,3.0,0.0,6.0,4.0,0.0,1.0,0.0,...,4.0,8.0,2.0,0.0,0.0,0.0,0.0,31.0,17.0,15.0


In [620]:
syn.to_csv('synthetic_data.csv', index=True)

In [621]:
plt.plot(syn['Total Row Score A'], "o", label="Total Row Score A")
plt.plot(syn['Total Row Score C'], "o", label="Total Row Score C")
plt.legend( loc='upper right')
